# Initial load

This notebook initially loads the datalets:
- nBew_Jakobcsv
- patientenId_Jakob
- sturzd_Jakob

Furthermore, the otebook processes (some of the) files and saves them as .pks files. This makes them faster to load. 




## Imports

In [1]:
import pandas as pd
import time
import csv
from datetime import datetime

## Functions

*csv_processer* is needed to process sturzd_Jakob, since the file uses ';' as separater but in some of the text boxes of the file, the same symbol is used in a text. Therefore this funciton takes the csv, and whenever something is in quotaiton, the processer changes the ';' to the inQuotesSep.

In [2]:
def csv_processer(inputFile, outputFile, inQuotesSep, sep=';'):
    # Open input CSV file for reading
    with open(inputFile, 'r') as f:
        # Create CSV reader object using specified delimiter
        reader = csv.reader(f, delimiter=sep)
        
        # Open output CSV file for writing
        with open(outputFile, 'w', newline='') as out:
            # Create CSV writer object using specified delimiter
            writer = csv.writer(out, delimiter=sep)
            
            # Iterate over each row in the input CSV file
            for row in reader:
                # Create a new row to store modified field values
                newRow = []
                
                # Iterate over each field in the current row
                for field in row:
                    #field = field.replace('"', '')
                    # If the separator character is found in the field
                    if sep in field:
                        # Replace the separator with specified character
                        field_new = field.replace(sep, inQuotesSep)
                        # Append modified field to the new row
                        newRow.append(field_new)
                    else:
                        # If separator character not found, append field to new row as is
                        newRow.append(field)
                
                # Write the new row to the output CSV file
                writer.writerow(newRow)

# Example usage:
# processer('input.csv', 'output.csv', '|')

## Processing

In [3]:
#preprocess and import sturzd
s= time.time()
csv_processer("data/src/sturzd_Jakob.csv","data/src/sturzd_clean.csv","|")
end1 =time.time()
print("processed sturzd in ", str(round(end1-s,2))," sec")

processed sturzd in  0.08  sec


## Imports

In [4]:
#load patID and nBew files
s =time.time()
patID = pd.read_csv("data/src/patientenId_Jakob.csv", delimiter=';')
end1 =time.time()
print("imported PatID in ",str(round(end1-s,2)), " sec")
nBew = pd.read_csv("data/src/nBew_Jakob.csv", delimiter=';')
end2 =time.time()
print("imported nBew in ",str(round(end2-end1,2)), " sec")

imported PatID in  1.75  sec
imported nBew in  10.48  sec


In [5]:
s = time.time()
fallD =pd.read_csv("data/src/sturzd_clean.csv", delimiter=';')
end =time.time()
print('Imported sturzd in ',str(round(end-s,2)),' sec')

Imported sturzd in  0.04  sec


# checking a couple of numbers

In [6]:
n_cases = patID.shape[0]
n_pat = patID['c_patnr_pseudonym'].nunique()
print('number of cases: ',n_cases)
print('number of patients: ', n_pat )
print('number of cases per patient: ', round(n_cases/n_pat,2))


number of cases:  3332640
number of patients:  932103
number of cases per patient:  3.58


In [7]:
n_movements = nBew.shape[0]
print('number of movements: ', n_movements)
print('number of movements per case: ', round(n_movements/n_cases,2))

number of movements:  10689561
number of movements per case:  3.21


In [8]:
n_fall = fallD.shape[0]
print('number of falls: ', n_fall)

number of falls:  14556


In [ ]:
nBew['c_orgfa'].nunique()

# Extract unique values
unique_values = nBew['c_orgfa'].unique()

# Convert the unique values to a DataFrame
unique_values_df = pd.DataFrame(unique_values, columns=['c_orgfa'])


# invstigation time Frame

In [10]:
# Convert date strings to datetime objects
date_start = datetime.strptime('2016-05-01', '%Y-%m-%d')
date_end = datetime.strptime('2022-04-30', '%Y-%m-%d')

### nBew 

filter all movements so that only the falls between 2016 and 2022 are in the dataset

In [11]:
#convert to datetime from UNIX timestamp
nBew['from_time_converted'] = nBew['from_time'].apply(datetime.fromtimestamp)
nBew['till_time_converted'] = nBew['till_time'].apply(datetime.fromtimestamp)

before = nBew.shape[0]


# Filter rows based on date range
nBew = nBew[(nBew['from_time_converted'] >= date_start) & (nBew['from_time_converted'] <= date_end)]
nBew = nBew[(nBew['till_time_converted'] >= date_start) & (nBew['till_time_converted'] <= date_end)]


after = nBew.shape[0]

print(f"Number of rows removed: {before - after}")
print(f"Number of rows remaining: {after}")
print(f"Percentage of rows remaining: {after / before * 100:.2f}%")

Number of rows removed: 1355228
Number of rows remaining: 9334333
Percentage of rows remaining: 87.32%


### sturzD


filter all falls so that only the falls between 2016 and 2022 are in the dataset

In [12]:
#convert to datetime from UNIX timestamp
fallD['Sturz_datetime_converted'] = fallD['Sturz_datetime'].apply(datetime.fromtimestamp)
before = fallD.shape[0]

# Filter rows based on date range
fallD = fallD[(fallD['Sturz_datetime_converted'] >= date_start) & (fallD['Sturz_datetime_converted'] <= date_end)]

after = fallD.shape[0]

fallD['Sturz_datetime_converted'] = pd.to_datetime(fallD['Sturz_datetime_converted'])

print(f"Number of rows removed: {before - after}")
print(f"Number of rows remaining: {after}")
print(f"Percentage of rows remaining: {after / before * 100:.2f}%")

Number of rows removed: 296
Number of rows remaining: 14260
Percentage of rows remaining: 97.97%


In [13]:
fallD['Hour'] = fallD['Sturz_datetime'].apply(lambda ts: datetime.fromtimestamp(ts).hour)


# Valididy checks

checks for validity and adjusts if needed. The follwoing should apply

**patID**: 
- c_pseudonym must be unique

**nBew**:
- for each c_pseudonym, the line times must not with from_time and till_time
- c_lfdnr must be increasing and without misses

**sturzD**: 
- Sturz_datetime not exactly twice
- Sturz_datetime within time of stay

### patID

In [14]:
patID_unique_ID= patID['c_pseudonym'].is_unique
print('patID is unique: ', patID_unique_ID)


patID is unique:  False


get not unique id

In [15]:
duplicates_ID = patID[patID.duplicated(subset=['c_pseudonym'])]
print('There are',duplicates_ID.shape[0], 'duplicates')
if duplicates_ID.shape[0]>0:
    print('-> there are ', duplicates_ID.shape[0],' cases that are associated to more then one patient')

There are 1 duplicates
-> there are  1  cases that are associated to more then one patient


In [16]:
# get all instances of that one duplicate
duplicated_ID = duplicates_ID['c_pseudonym'].tolist()[0]
duplicates = patID[patID['c_pseudonym']==duplicated_ID]

In [17]:
# check if the patient number that is used in the duplicates is unique, if only 1 then unique
for pseudonym in duplicates['c_patnr_pseudonym']:
    count = patID[patID['c_patnr_pseudonym'] == pseudonym]['c_patnr_pseudonym'].count()
    print(f"{pseudonym}: {count}")

350670: 1
769976: 1


while there are two cases with the same case number, the corresponding patient (two distinct patiens) each only exist once. therefore, to fix the prblem we can just eliminate one of the duplicate.

this only works, since patient number is nowere else used.

In [18]:
#droping duplicates
patID = patID.drop_duplicates(subset=['c_pseudonym'])

In [19]:
#check for duplicates
patID_unique_ID= patID['c_pseudonym'].is_unique
print('patID is unique: ', patID_unique_ID)


patID is unique:  True


No duplicates anymore-> perfect.

# Filter to have only the cases that are in the movement

Add Patient ID to 


In [20]:
helper_nBew = nBew['c_pseudonym'].copy()
helper_nBew = nBew['c_pseudonym'].copy().drop_duplicates()
helper_nBew = helper_nBew.reset_index()
helper_nBew['is_in_nBew'] = True
helper_nBew = helper_nBew.drop(columns=['index'])

In [21]:
# Calculate the number of rows before filtering
before = patID.shape[0]

# Merge patID with helper_nBew
patID = patID.merge(helper_nBew, on='c_pseudonym', how='left')

# Filter patID where is_in_nBew is True
patID = patID[patID['is_in_nBew'] == True].drop(columns=['is_in_nBew'])

# Calculate the number of rows after filtering
after = patID.shape[0]


print(f"Number of rows removed: {before - after}")
print(f"Number of rows remaining: {after}")
print(f"Percentage of rows remaining: {after / before * 100:.2f}%")


Number of rows removed: 407775
Number of rows remaining: 2924864
Percentage of rows remaining: 87.76%


delete all patients from patientID that are not moving (blanks, something wrong, since admission is a movement)

# Adjust specialitites to ignore different campus

In [23]:
specialities = pd.read_pickle("data/specialities/specialities.pkl")

In [24]:
specialities_mapping = pd.read_pickle('data/specialities/speciality_mapping.pkl')

In [25]:
spec = pd.merge(specialities, specialities_mapping, left_on='speciality', right_on='German', how='left').drop_duplicates(subset=['OEs'])

In [26]:
spec.drop(columns=['speciality','German','English'], inplace=True)
spec.rename(columns={'Adjusted Category':'speciality'}, inplace=True)


## temporary part start

In [27]:
n_wards = spec['OEs'].nunique()
n_departments = spec['speciality'].nunique()

print('number of wards: ', n_wards)
print('number of departments: ', n_departments)

number of wards:  1510
number of departments:  81


In [28]:
temp = pd.merge(unique_values_df, spec, left_on='c_orgfa', right_on='OEs', how='left')
temp.shape[0]

213

In [29]:
temp['speciality'].nunique()

60

## temporary part end

In [30]:
nBew_temp = pd.merge(nBew, spec, left_on='c_orgfa', right_on='OEs', how='left')
nBew_temp = nBew_temp.drop(columns=['OEs'])
nBew_temp = nBew_temp.rename(columns={'c_orgfa': 'c_orgfa_org'})
nBew_temp = nBew_temp.rename(columns={'speciality': 'c_orgfa'})
nBew = nBew_temp

In [ ]:
fallD_temp = pd.merge(fallD, spec, left_on='Fachliche Zuweisung', right_on='OEs', how='left')
fallD_temp = fallD_temp.drop(columns=['OEs'])
fallD_temp = fallD_temp.rename(columns={'Fachliche Zuweisung': 'Fachliche Zuweisung Alt'})
fallD_temp = fallD_temp.rename(columns={'speciality': 'Fachliche Zuweisung'})
fallD_temp['Fachliche Zuweisung'] = fallD_temp['Fachliche Zuweisung'].fillna('Unknown')
fallD = fallD_temp



## File storing as PKL

In [ ]:
#store all files in .pkl
patID.to_pickle("data/src_pkl/patID.pkl")
nBew.to_pickle("data/src_pkl/nBew.pkl")
fallD.to_pickle("data/src_pkl/sturzD.pkl")